## Generation of the GT for Training

In [2]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
from scipy import spatial

from tqdm.auto import tqdm

from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
sem_clouds_filename = f"{export_ds}/sem_clouds.npy"

print(f"Loading from sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /mnt/data/datasets/nuscenes/processed/sem_clouds.npy
Shape of sem clouds is (850, 2, 200, 200)


In [21]:
n_clouds = sem_cloud_features.shape[0]
classes = [None] * n_clouds
for i in range(0, n_clouds):
    sem_cloud = np.reshape(sem_cloud_features[i,:,:,:], (2, -1)).T    
    classes[i] = sem_cloud[:,1]
classes = np.array(classes[:])
print(f"shape of classes is {classes.shape}")
n_unique_classes = len(np.unique(classes))
print(f"we have {n_unique_classes} classes in the dataset.")

shape of classes is (850, 40000)
we have 32 classes in the dataset.
